In [5]:
import numpy as np
data = np.load('C:\\Users\\zihanlin\\Desktop\\File\\Research\\npz_data\\bryankelly_without_missing.npz')
processed_data = data['processed_data']
T, N, M = processed_data.shape
returns = processed_data[:, :, 1]
fama_french_data= np.load('C:\\Users\\zihanlin\\Desktop\\File\\Research\\npz_data\\riskfreerate.npz')
fama_french_data= fama_french_data['fama_french_data']
r_f= fama_french_data[:,4]
r_f= r_f/100
date= fama_french_data[:,0]
date_start= np.asscalar(np.argwhere(date==198406))
date_end= np.asscalar(np.argwhere(date==201405))
returns= returns- (r_f[date_start:date_end+1:,np.newaxis])
# The dimension of returns: T*N
chara_data = processed_data[:, :, 5:9]
# chara_data = processed_data[:, :, [8, 17, 20, 33, 37]]
# chara_data= processed_data[:,:,[8,17, 18 ,20,33, 37]]
# chara_data= processed_data[:,:,[8,17,33,37]]
T, N= returns.shape
def get_tree(depth, res, nodetag,  value_array, L, d, chara_data, oldlist, parent):
    if depth == d:
        final_array = np.ones([N, 1])
        for k in range(L):
            temp = value_array[:, k].reshape([N, 1])
            final_array = final_array * temp
        # print(np.sum(final_array))
        if np.sum(final_array) < 10 ** (-3):
            print('An error over here')
        final_array = 1.0 * final_array / np.sum(final_array)
        # It alrady gets updated.
        # if np.sum(final_array)<10**(-3):
        #     print (value_array)
        res.append([final_array]) 
        nodetag.append(oldlist)
    else:
        for i in range(L):
            for j in range(2):
                if len(parent) != 0:
                    index = np.ones((N), dtype=bool)
                    for k in parent:
                        index = (index) & ((value_array[:, k] != 0))
                    data_median = np.median(chara_data[i, index])
                else:
                    index = (value_array[:, i] != 0)
                    data_median = np.median(chara_data[i, index])
                temp_array = value_array.copy()
                newlist= list(oldlist)
                newlist+=[(i,j)]
                parent.append(i)
                if j == 0:
                    temp_array[(index) & (chara_data[i, :] <= data_median), i] = 0
                    get_tree(depth + 1, res, nodetag, temp_array, L, d, chara_data, newlist, parent)
                elif j == 1:
                    temp_array[(index) & (chara_data[i, :] >= data_median), i] = 0
                    get_tree(depth + 1, res, nodetag, temp_array, L, d, chara_data, newlist, parent)
                parent.pop()
L= chara_data.shape[2]
d= 4
portfolio = np.zeros((T - 1, (2*L)**d))
for i in range(T-1):
# for i in range(0,1):
    if i%20==0:
            print ('Have gone through %d' %(i))
    # print ('Have gone through %d' %(i))
    res = []
    nodetag= []
# The first parameter is number of characteristics, and the second is the depth of the tree.
    get_tree(0, res, nodetag, np.ones([N, L]), L, d, chara_data[i,:].T, [], [])
    # output = np.array(res[0][0])
    # output = np.reshape(output, [N, 1])
    # for j in range(1, len(res)):
    #     output = np.append(output, np.array(res[j][0]), axis=1)
    # portfolio[i, :] = returns[i + 1, :].dot(output)
    for j in range(len(res)):
        portfolio[i][j]= returns[i+1,:].dot(res[j])


Have gone through 0
Have gone through 20
Have gone through 40
Have gone through 60
Have gone through 80
Have gone through 100
Have gone through 120
Have gone through 140
Have gone through 160
Have gone through 180
Have gone through 200
Have gone through 220
Have gone through 240
Have gone through 260
Have gone through 280
Have gone through 300
Have gone through 320
Have gone through 340


In [23]:
D= 10
portfolio2= np.zeros((T-1, L*D))
for i in range(T-1):
    for j in range(L):
        chara_data_t= chara_data[i,:,j]
        index= np.argsort(chara_data_t)
        # print (index)
        length= int(len(index)/D)
        # print (length)
        for k in range(D):
            index_k= index[int(length*k):int(length*(k+1))]
            portfolio2[i, j*D+k]= np.mean(returns[i+1,index_k ])
portfolio= np.concatenate((portfolio1, portfolio2), axis= 1)

In [17]:

import scipy.linalg as la
M= portfolio.shape[1]
K = 4
window= 120
optimal_return = np.zeros((T-1-window))
gamma = 100
for t in range(T-1-window):
# for t in range(0,1):
    if t % 20 == 0:
        print('Have gone through iteration %d' % (t))
    portfolio_t = portfolio[t:(t + window), :]
    covmat_t = portfolio_t.T.dot(portfolio_t) 
    portfolio_mean_t = np.sum(portfolio_t, axis=0, keepdims= True)
    # print (portfolio_mean_t.shape)
    covmat_t = covmat_t + portfolio_mean_t.T.dot(portfolio_mean_t) / window * gamma
    # variance_t, loading_t = np.linalg.eig(covmat_t)
    variance_t, loading_t= la.eigh(covmat_t, eigvals=(M-K, M-1))
    Factor_t= portfolio_t.dot(loading_t)
    mu= np.mean(Factor_t, axis= 0)
    covmat_factor= np.cov(Factor_t.T)
    weight= np.linalg.inv(covmat_factor).dot(mu)
    optimal_return[t] = (portfolio[t + window, :].dot(loading_t)).dot(weight)
mean_optimal= np.mean(optimal_return)
cov_optimal= np.cov(optimal_return)
print (abs(mean_optimal)/np.sqrt(cov_optimal))

Have gone through iteration 0


KeyboardInterrupt: 

In [38]:
# In sample sharpe ratio
gamma= 100
# portfolio_demean = portfolio - np.mean(portfolio, axis=0)
# Dimension of portfolio: T*M
covmat = portfolio.T.dot(portfolio) 
portfolio_mean= np.sum(portfolio, axis= 0, keepdims=True)
covmat= covmat+ portfolio_mean.T.dot(portfolio_mean)/(T-1)*gamma
variance, loading = np.linalg.eig(covmat)
output= []
for K in range(1, 20):
    loading_chosen = loading[:, :K]
    Factor = portfolio.dot(loading_chosen)
    mu = np.mean(Factor, axis=0)
    covmat_factor = np.cov(Factor.T)
    if K==1:
        output.append (abs(mu)*covmat_factor**(-1/2))
    else:
        output.append(np.sqrt(np.dot(np.dot(mu, np.linalg.inv(covmat_factor)), mu.T)))
twodecimals = ["&%.2f" % np.real(v) for v in output]
print (''.join(twodecimals))


&0.23&0.26&0.31&0.39&0.42&0.43&0.44&0.45&0.48&0.48&0.50&0.51&0.55&0.55&0.55&0.55&0.55&0.55&0.58


In [12]:
# In sample code for ultra dimensional
import scipy.linalg as la
K =20
gamma =100
M= portfolio.shape[1]
covmat = portfolio.T.dot(portfolio) 
portfolio_mean= np.sum(portfolio, axis= 0, keepdims=True)
covmat= covmat+ portfolio_mean.T.dot(portfolio_mean)/(T-1)*gamma
variance, loading= la.eigh(covmat, eigvals=(M-K, M-1))
Factor= portfolio.dot(loading)
output= []
# Factor = portfolio.dot(loading_chosen)/N
for i in range(K-1,-1,-1):
    mu = np.mean(Factor[:,i:K], axis=0)
    covmat_factor = np.cov(Factor[:,i:K].T)
    if i==(K-1): 
        output.append (abs(mu)*covmat_factor**(-1/2))
    else:
        output.append(np.sqrt(np.dot(np.dot(mu, np.linalg.inv(covmat_factor)), mu.T)))
twodecimals = ["&%.2f" % np.real(v) for v in output]
print (''.join(twodecimals))

19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
&0.23&0.26&0.31&0.39&0.42&0.43&0.44&0.45&0.48&0.48&0.50&0.51&0.55&0.55&0.55&0.55&0.55&0.55&0.58&0.58


In [6]:
# The case for ultra high dimensional 
portfolio= portfolio1
import scipy.linalg as la
K = 5
window= 120
optimal_return = np.zeros((T-window))
gamma = 100
T, M = portfolio.shape
ones_T= np.ones((window, 1))
# Maybe can just change the T-1 to be T.
for t in range(T-window):
# for t in range(0,1):
    if t % 20 == 0:
        print('Have gone through iteration %d' % (t))
    portfolio_t = portfolio[t:(t + window), :]
    covmat_t= portfolio_t.dot(portfolio_t.T).dot(gamma/T* ones_T.dot(ones_T.T))
    covmat_t= covmat_t+ portfolio_t.dot(portfolio_t.T)
    # variance_t, Factor_t = np.linalg.eig(covmat_t)
    variance_t, Factor_t= la.eigh(covmat_t, eigvals=(window-K, window-1))
    Factor_t= Factor_t[:,:K]
    loading_t= Factor_t.T.dot(portfolio_t)
    mu= np.mean(Factor_t, axis= 0)
    covmat_factor= np.cov(Factor_t.T)
    weight= np.linalg.inv(covmat_factor).dot(mu)
    optimal_return[t] = (portfolio[t + window, :].dot(loading_t.T)).dot(weight)
mean_optimal= np.mean(optimal_return)
cov_optimal= np.cov(optimal_return)
print (abs(mean_optimal)/np.sqrt(cov_optimal))

Have gone through iteration 0
Have gone through iteration 20
Have gone through iteration 40
Have gone through iteration 60
Have gone through iteration 80
Have gone through iteration 100
Have gone through iteration 120
Have gone through iteration 140
Have gone through iteration 160
Have gone through iteration 180
Have gone through iteration 200
Have gone through iteration 220
0.253768467093


In [3]:
# Get the pricing error
portfolio=  portfolio1
import scipy.linalg as la
K = 4
window= 120
gamma = 100
T, M = portfolio.shape
pricing_error= np.zeros((T-window,M ))
ones_T= np.ones((window, 1))
for t in range(T-window):
# for t in range(0,1):
    if t % 20 == 0:
        print('Have gone through iteration %d' % (t))
    portfolio_t = portfolio[t:(t + window), :]
    # covmat_t = portfolio_t.T.dot(portfolio_t) 
    # portfolio_mean_t = np.sum(portfolio_t, axis=0, keepdims= True)
    # # print (portfolio_mean_t.shape)
    # covmat_t = covmat_t + portfolio_mean_t.T.dot(portfolio_mean_t) / window * gamma
    covmat_t= portfolio_t.dot(portfolio_t.T).dot(gamma/T* ones_T.dot(ones_T.T))
    covmat_t= covmat_t+ portfolio_t.dot(portfolio_t.T)
    variance_t, Factor_t = np.linalg.eig(covmat_t)
    Factor_t= Factor_t[:,:K]
    # variance_t, loading_t = np.linalg.eig(covmat_t)
    # variance_t, Factor_t= la.eigh(covmat_t, eigvals=(window-K, window-1))
    loading_t= Factor_t.T.dot(portfolio_t)
    # loading_t: K*M.
    # Factor_t= portfolio_t.dot(loading_t)
    pricing_error[t,:] = (portfolio[t + window, :]- portfolio[t+window, :].dot(loading_t.T).dot(np.linalg.inv(loading_t.dot(loading_t.T))).dot(loading_t))
avg_pricing_error= np.mean(pricing_error, axis= 0)
print (np.sqrt(np.sum(avg_pricing_error**2)/M))

Have gone through iteration 0
Have gone through iteration 20
Have gone through iteration 40
Have gone through iteration 60
Have gone through iteration 80
Have gone through iteration 100
Have gone through iteration 120
Have gone through iteration 140
Have gone through iteration 160
Have gone through iteration 180
Have gone through iteration 200
Have gone through iteration 220
0.00136893767097


In [ ]:
# In sample pricing error
import scipy.linalg as la
T,M= portfolio.shape
K= 5
gamma= 100
ones_T= np.ones((T,1))
# portfolio_demean = portfolio - np.mean(portfolio, axis=0)
# Dimension of portfolio: T*M
# covmat = portfolio.T.dot(portfolio) 
# portfolio_mean= np.sum(portfolio, axis= 0, keepdims=True)
# covmat= covmat+ portfolio_mean.T.dot(portfolio_mean)/(T-1)*gamma
# variance, loading = la.eigh(covmat, (M-K, M-1))
covmat= portfolio.dot(portfolio.T).dot(gamma/T* ones_T.dot(ones_T.T))
covmat= covmat+ portfolio.dot(portfolio.T)
    # variance_t, loading_t = np.linalg.eig(covmat_t)
variance, Factor= la.eigh(covmat, eigvals=(T-K, T-1))
loading= Factor.T.dot(portfolio)
pricing_error= portfolio- Factor.dot(loading)
print (np.sqrt(np.sum(pricing_error**2)/M)/(T))

[ 0.1180668   0.24490528  0.24922966  0.25027935  0.25107852  0.25027954
  0.25099506  0.25016822  0.25111488  0.25139332  0.25216388  0.25247373
  0.2527811   0.25305591  0.25326001  0.25356122  0.25352739  0.25376014
  0.25347647  0.25361371]


In [7]:
def compute_sharpe(K):
    import scipy.linalg as la
    T, M = portfolio.shape
    window= 120
    optimal_return = np.zeros((T-window))
    gamma = 100    
    ones_T= np.ones((window, 1))
# Maybe can just change the T-1 to be T.
    for t in range(T-window):
# for t in range(0,1):
#         if t % 20 == 0:
#             print('Have gone through iteration %d' % (t))
        portfolio_t = portfolio[t:(t + window), :]
        covmat_t= portfolio_t.dot(portfolio_t.T).dot(gamma/T* ones_T.dot(ones_T.T))
        covmat_t= covmat_t+ portfolio_t.dot(portfolio_t.T)
    # variance_t, Factor_t = np.linalg.eig(covmat_t)
        variance_t, Factor_t= la.eigh(covmat_t, eigvals=(window-K, window-1))
        Factor_t= Factor_t[:,:K]
        loading_t= Factor_t.T.dot(portfolio_t)
        mu= np.mean(Factor_t, axis= 0)
        covmat_factor= np.cov(Factor_t.T)
        if K>=2:
            weight= np.linalg.inv(covmat_factor).dot(mu)
        else:
            weight= 1
        optimal_return[t] = (portfolio[t + window, :].dot(loading_t.T)).dot(weight)
    mean_optimal= np.mean(optimal_return)
    cov_optimal= np.cov(optimal_return)
    return (abs(mean_optimal)/np.sqrt(cov_optimal))
K= 20
optimal_sharpe= np.zeros((K))
for i in range(K):
    optimal_sharpe[i]= compute_sharpe(i+1)
print (optimal_sharpe)

[ 0.1156334   0.24708772  0.25203361  0.25343729  0.25433099  0.25384069
  0.2543155   0.25356588  0.2544314   0.25500979  0.25571067  0.25565999
  0.25615806  0.25674477  0.25663045  0.2567508   0.25690141  0.25693231
  0.25671441  0.25681932]


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
df2 = pd.DataFrame(sharpe_ratio, columns=['1 Factor','2 Factor','3 Factor','4 Factor','5 Factor','6 Factor','7 Factor'], index= ['P1', 'P2'])
df2.plot.bar()
plt.savefig('C://Users//zihan//Desktop//Files//Research//sharpe_tree_decile.png')
plt.clf()

In [7]:
path_target = 'C:/Users/zihanlin/Desktop/File/Research/npz_data/small_tree_portfolio.npz'
print('Saving data to ' + path_target)
np.savez(path_target, portfolio=portfolio)
print('Finished! ')

Saving data to C:/Users/zihanlin/Desktop/File/Research/npz_data/small_tree_portfolio.npz
Finished! 
